In [ ]:
import logging
import json
import os
import requests
import time
import traceback
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import pygeohash as pgh
import folium
import seaborn as sns
import matplotlib.colors as mcolors
import ast
import math
import numpy as np
import geopy.distance
import itertools

In [ ]:
import os
os.chdir('/Users/apple/Desktop/bmtc_routes_data')
os.getcwd()

**Loading Data**

In [ ]:
#Obtain trips data
trips_raw = pd.read_csv('trips.txt', delimiter=',')
routes_raw = pd.read_csv('routes.txt', delimiter=',')
stops_raw = pd.read_csv('stops.txt', delimiter=',')
stop_times_raw = pd.read_csv('stop_times.txt', delimiter=',')

**What we need is**

__1.__ stop_name, stop_lat, stop_lon, trip_count  <br>
__2.__ route_name, route_id, direction_id, sequence_of_stops, trip_count  <br>

In [ ]:
stops_agg = stops_raw[['stop_name', 'stop_id', 'stop_lat', 'stop_lon']].merge(
    stop_times_raw[['stop_id', 'trip_id']], on = 'stop_id', how = 'inner').merge(
    trips_raw[['trip_id']], on = 'trip_id', how = 'inner'
).groupby('stop_name').agg({'stop_lat': 'mean', 'stop_lon': 'mean', 'trip_id': 'nunique'}).reset_index(drop=False).copy(deep=True)

stops_agg.columns = ['stop_name', 'lat', 'lng', 'trips']
stops_agg.to_csv('stops_agg.csv', index=False)
stops_agg.head(5)

In [ ]:
routes_all = routes_raw[['route_id', 'route_short_name']].merge(
    trips_raw[['route_id', 'trip_id', 'direction_id']], on = 'route_id', how = 'inner').merge(
    stop_times_raw[['trip_id', 'stop_id', 'stop_sequence']], on = 'trip_id', how = 'inner').merge(
    stops_raw[['stop_id', 'stop_name']], on = 'stop_id', how = 'inner'
).copy(deep=True)
# [routes_all.route_short_name.str.contains('500', case=False)]

routes_agg = routes_all.groupby(['route_id','route_short_name']).agg({
    'trip_id': 'nunique'}).reset_index(drop=False).merge(
    routes_all[['route_short_name', 'direction_id', 
                'stop_sequence', 'stop_name']].drop_duplicates().sort_values(by=[
    'route_short_name', 'direction_id', 'stop_sequence']).groupby([
    'route_short_name', 'direction_id'])['stop_name'].agg(list).reset_index(drop=False),
    on = ['route_short_name'], how = 'inner' 
).reset_index(drop=True).copy(deep=True)

routes_agg.columns = ['route_id', 'route_short_name', 'trips', 'direction_id', 'stop_list']
routes_agg.to_csv('routes_agg.csv', index=False)
routes_agg.head(5)

In [ ]:
# Load stops data
stops_df = stops_agg.copy(deep=True)
stops_df['lat_lng'] = stops_df.apply(lambda x: (x['lat'], x['lng']), axis=1)
stops_df.head(3)

In [ ]:
# Load routes data
routes_df = routes_agg.copy(deep=True)
routes_df.head(3)

In [ ]:
# Bengaluru center - center of all bus stops
bengaluru_center = (np.mean(stops_df.lat), np.mean(stops_df.lng))
bengaluru_center

### Mapping Stops by their level of activity

- For a stop to be active, a fair expectation can be set of having a bus arrive on an average of every 20 minutes or sooner (assume an average of one bus every 10 mins during earthly hours - 8am to 8pm)
- That makes it an average of 72 trips/day

- We define stops in the following manner:  <br>
_1._ Active Stops -> >= 72 trips/day # A bus every 20 mins or more frequently <br>
_2._ Infrequent Stops -> 72 > trips/day >= 24 # A bus every hour to every 20 mins <br>
_3._ Dormant Stops -> < 24 trips/day # Less than one bus every hour <br>

- They will me mapped Green, Yellow, Red respectively

In [ ]:
palette = "RdYlGn"
p = sns.color_palette(palette, 3).as_hex()
p#[0]

stops_df['stop_freq_code'] = stops_df.apply(
    lambda x: 'dormant' if x['trips'] < 24 else 'infrequent' if x['trips'] < 72 else 'active', axis=1)
stops_df['stop_freq_color'] = stops_df.apply(
    lambda x: p[0] if x['trips'] < 24 else p[1] if x['trips'] < 72 else p[2], axis=1)

m = folium.Map(location=bengaluru_center, zoom_start=10)

for i in set(stops_df['stop_freq_color']):
    
    stop_locations = stops_df.loc[stops_df.stop_freq_color==i].copy(deep=True)
    
    for j, r in stop_locations[['lat', 'lng', 'stop_name']].iterrows():
        folium.Circle(
            location=[float(r['lat']), float(r['lng'])],
            radius=20,  # Radius in meters
            color=i,
            fill=True,
            fill_color=i,
            popup=r['stop_name']
        ).add_to(m)

# Display the map
m

### Mapping Limits of Bengaluru

In [ ]:
polygon_city = [(12.840555555555556, 77.67611111111111), (12.855833333333333, 77.66444444444446), (12.865, 77.63611111111112), (12.845555555555556, 77.59305555555555), (12.844722222222222, 77.55611111111111), (12.867777777777778, 77.49666666666667), (12.901944444444444, 77.46666666666667), (12.994444444444444, 77.4675), (13.057500000000001, 77.4738888888889), (13.088333333333335, 77.49277777777777), (13.126111111111111, 77.55222222222221), (13.131388888888889, 77.57472222222222), (13.129722222222222, 77.61611111111111), (13.114722222222222, 77.68805555555556), (13.0875, 77.73944444444444), (13.052222222222223, 77.77333333333333), (12.986666666666666, 77.7863888888889), (12.886666666666667, 77.74527777777777), (12.856388888888889, 77.71194444444444), (12.840555555555556, 77.67611111111111)]

# Define the coordinates of the polygon vertices
polygon_coords = polygon_city.copy()
# Create a map centered around the mean of the coordinates
map_center = [np.mean([lat for lat, long in polygon_coords]), np.mean([long for lat, long in polygon_coords])]
m = folium.Map(location=map_center, zoom_start=11)

# Add the polygon to the map
folium.Polygon(locations=polygon_coords, color='orange', fill=True).add_to(m)
c=0
# Add a marker for each coordinate, with a popup showing the latitude and longitude
for lat, lon in polygon_coords[:-1]:  # Exclude the last point as it's same as the first
    c=c+1
    folium.Marker([round(lat,4), round(lon,4)], popup=f'{c}: ({round(lat,4)}, {round(lon,4)})').add_to(m)

# Display the map
m


### Get all Geohashes within the polygon

In [ ]:
def get_geohashes_in_polygon(polygon, precision):
    # Find the bounding box of the polygon
    min_lat = min(polygon, key=lambda x: x[0])[0]
    max_lat = max(polygon, key=lambda x: x[0])[0]
    min_lon = min(polygon, key=lambda x: x[1])[1]
    max_lon = max(polygon, key=lambda x: x[1])[1]

    # Start with an arbitrary point in the bounding box
    current_point = (min_lat, min_lon)

    # Store geohashes in a set to avoid duplicates
    geohashes = set()

    while current_point[1] <= max_lon:
        while current_point[0] <= max_lat:
            # Generate geohash for the current point
            geohash = pgh.encode(current_point[0], current_point[1], precision=precision)
            geohashes.add(geohash)

            # Move north by approximately one geohash cell
            current_point = geopy.distance.distance(kilometers=geohash_size_km(precision)['vert']).destination(current_point, bearing=0)

        # Move east by one cell and reset latitude
        current_point = geopy.distance.distance(kilometers=geohash_size_km(precision)['horiz']).destination((min_lat, current_point[1]), bearing=90)
        current_point = (min_lat, current_point[1])

    # Filter geohashes that are actually inside the polygon
    return [gh for gh in geohashes if is_geohash_in_polygon(gh, polygon, precision)]

def geohash_size_km(precision):
    # Approximate sizes of geohash cells at different precisions in kilometers
    sizes_h = [5000, 1250, 156, 39, 4.9, 1.2, 0.140, 0.038, 0.0048, 0.0012] # horizontal
    sizes_v = [5000, 1250/2, 156, 39/2, 4.9, 1.2/2, 0.140, 0.038/2, 0.0048, 0.0012/2] # vertical
    return {'vert': sizes_v[precision - 1], 'horiz': sizes_h[precision - 1]}

def is_geohash_in_polygon(gh, polygon, precision):
    # Check if the center of a geohash is inside the polygon
    lat, lon = pgh.decode_exactly(gh)[:2]
    return point_in_polygon((lat, lon), polygon)

def point_in_polygon(point, polygon):
    # A simple algorithm to check if a point is inside a polygon
    # Note: This is for demonstration purposes and may not handle complex polygons or edge cases accurately.
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

bangalore_geohash_6 = pd.DataFrame(get_geohashes_in_polygon(polygon_city, precision = 6))
bangalore_geohash_6.columns = ['geohash6']
bangalore_geohash_7 = pd.DataFrame(get_geohashes_in_polygon(polygon_city, precision = 7))
bangalore_geohash_7.columns = ['geohash7']

### Get center and bounding box of geohashes

In [ ]:
# Function to get the bounding box corners of a geohash
def geohash_to_polygon(ghash):
    coords = pgh.decode_exactly(ghash)
    center = (coords[0], coords[1])
    north = coords[0]+coords[2]
    south = coords[0]-coords[2]
    east = coords[1]+coords[3]
    west = coords[1]-coords[3]
    
    return [(south, west), (north, west), (north, east), (south, east)]

bangalore_geohash_6['center'] = bangalore_geohash_6.geohash6.apply(lambda x: (pgh.decode_exactly(x)[0], pgh.decode_exactly(x)[1]))
bangalore_geohash_7['center'] = bangalore_geohash_7.geohash7.apply(lambda x: (pgh.decode_exactly(x)[0], pgh.decode_exactly(x)[1]))
bangalore_geohash_6['bbox'] = bangalore_geohash_6.geohash6.apply(geohash_to_polygon)
bangalore_geohash_7['bbox'] = bangalore_geohash_7.geohash7.apply(geohash_to_polygon)
bangalore_geohash_6.head(3)

### Get minimum distance of an active stop from a geohash

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8 # this is in kilometers.  For Earth radius in miles use 3959.87433 miles
#     R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km
    
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c * 1000 # distance in meters

# Find areas in Bangalore 7 Geohashes 1km or more away from Active stops
def stops_closest(loc, stops_list):
    return (min([haversine(loc[0], loc[1], s[0], s[1]) for s in stops_list]))

#stops_closest((1,2), 1)
bangalore_geohash_7['closest_active_stop_dist'] = bangalore_geohash_7.center.apply(lambda x: stops_closest(
    x, stops_list = list(stops_df[stops_df['stop_freq_code']=='active']['lat_lng'])))
bangalore_geohash_7.head(5)

#### Mapping Areas based on distance from the nearest active stop (haversine distance taken for simplicity)
- Green - within 500m 
- Yellow - 500m to 1km 
- Red - beyond 1km

In [ ]:
palette = "RdYlGn_r"
p = sns.color_palette(palette, 3).as_hex()
p#[0]

bangalore_geohash_7['closest_band'] = bangalore_geohash_7.apply(
    lambda x: 'a. within 500m' if x['closest_active_stop_dist'] < 500 
    else 'b. within 1km' if x['closest_active_stop_dist'] < 1000 else 'c. beyond 1km', axis=1)
bangalore_geohash_7['closeness_color'] = bangalore_geohash_7.apply(
    lambda x: p[0] if x['closest_active_stop_dist'] < 500 
    else p[1] if x['closest_active_stop_dist'] < 1000 else p[2], axis=1)


In [ ]:
# Create a map centered around the first geohash
m = folium.Map(location=bengaluru_center, zoom_start=11)

# Plot each geohash
for index, row in bangalore_geohash_7.iterrows():
    polygon = geohash_to_polygon(row['geohash7'])
    folium.Polygon(locations=polygon, color=row['closeness_color'], fill=True, 
                   fill_opacity=0.4, weight=0).add_to(m)

# Display the map
m

##### It can be seen that a few important areas like Bellandur-Varthur, Harlur and arterial roads from Sarjapur Road are not covered well by BMTC

### Next, we look at 1 stop Reachability

In [ ]:
# First, we look at all possible combinations of trips
routes_temp = []
for index, row in routes_df.iterrows():
    stop_combinations = list(itertools.combinations(row['stop_list'],2))
    for i in stop_combinations:
        routes_temp.append({'route': row['route_short_name'], 'direction': row['direction_id'], 
                            'from_stop': i[0], 'to_stop': i[1], 'trips': row['trips']})

routes_combinations = pd.DataFrame(routes_temp)
routes_combinations[routes_combinations.route=='500-CA'].head(5)

##### We shortlist pairs of stop combinations with >= 72 trips between them

In [ ]:
direct_connectivity_df = routes_combinations[['from_stop','to_stop','trips']].groupby([
    'from_stop','to_stop']).agg({'trips': 'sum'}).reset_index().groupby([
    'from_stop','to_stop']).filter(lambda x: x['trips'].sum() >= 72).reset_index(drop=True)
direct_connectivity_df.head(5)

In [ ]:
onestop_connectivity_df = direct_connectivity_df[['from_stop','to_stop']].merge(direct_connectivity_df,
                                                                                left_on = ['to_stop'],
                                                                                right_on = ['from_stop'],
                                                                                suffixes = ['_1', '_2']
                                                                               ).copy(deep=True)

onestop_connectivity_df = onestop_connectivity_df[
    onestop_connectivity_df.from_stop_1 != onestop_connectivity_df.to_stop_2].reset_index(drop=True).copy(deep=True)
onestop_connectivity_df.head(5)

##### Get all geohashes within a fixed radius of a stop

In [ ]:
# Works for geohash 8 or below(<=8)
def get_geohashes(lat_long, reachable_radius = 0.5, precision = 8):
    try:
        earth_circumference_km = 40075
        # Convert original latitude longitude to radians
        lat_rad = math.radians(lat_long[0]) 
        long_rad = math.radians(lat_long[1])

        # Calculate the change in latitude and longitude in radians
        delta_rad = (reachable_radius / earth_circumference_km) * 2 * math.pi
        lat_max, lat_min, long_max, long_min = math.degrees(lat_rad+delta_rad), math.degrees(lat_rad-delta_rad), math.degrees(long_rad+delta_rad), math.degrees(long_rad-delta_rad)

        geohashes = set()
        distance_gh = 0.015
        earth_circumference_km = 40075
        rad_movement = (distance_gh / earth_circumference_km) * 2 * math.pi
        lat_set, long_set = lat_min, long_min

        while lat_set < lat_max:
            
            while long_set < long_max:
                if haversine(lat_set, long_set, lat_long[0], lat_long[1]) <= reachable_radius*1000:
                    geohash = pgh.encode(lat_set, long_set, precision=precision)
                    geohashes.add(geohash)
                # Move east
                long_set = long_set + math.degrees(rad_movement)
            # Move north
            lat_set = lat_set + math.degrees(rad_movement)
            long_set = long_min
        return geohashes
    except:
        print(lat_long[0], lat_long[1], reachable_radius, precision)

geohashes = get_geohashes([12.91644, 77.63458], reachable_radius = 0.5, precision = 8)
# print(geohashes)
# List of geohashes to plot

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
stops_df['loc_geohashes'] = stops_df.progress_apply(lambda x: get_geohashes(x['lat_lng'], precision=7) , axis=1)

In [ ]:
def get_lat_long_and_distance(from_stop, to_stop, stops_data = stops_df):
    try:
        from_loc = list(stops_data[stops_data['stop_name'] == from_stop].iloc[0]['lat_lng'])
        to_loc = list(stops_data[stops_data['stop_name'] == to_stop].iloc[0]['lat_lng'])
        to_gh = list(stops_data[stops_data['stop_name'] == to_stop].iloc[0]['loc_geohashes'])

        return ({'from_loc': from_loc, 'to_loc': to_loc, 'to_gh': to_gh})
    except:
        print (from_stop, to_stop)#, from_loc, to_loc)
        return ({'from_loc': 99.99, 'to_loc': 99.99, 'to_gh': 'aaaaaaaa'})

direct_connectivity_df['locations'] = direct_connectivity_df.progress_apply(lambda x: get_lat_long_and_distance(x['from_stop'], 
                                                                                          x['to_stop']
                                                                                         ), axis = 1)
direct_connectivity_df = pd.concat([direct_connectivity_df.drop(['locations'], axis=1),
                                    pd.json_normalize(direct_connectivity_df['locations'])], axis=1
                                  )
direct_connectivity_df.head(5)


In [ ]:
# onestop_connectivity_df['locations'] = onestop_connectivity_df.progress_apply(
#     lambda x: get_lat_long_and_distance(x['from_stop_1'], x['to_stop_2']), axis = 1)
# onestop_connectivity_df = pd.concat([onestop_connectivity_df.drop(['locations'], axis=1),
#                                     pd.json_normalize(onestop_connectivity_df['locations'])], axis=1
#                                   )
# onestop_connectivity_df.head(5)

In [ ]:
stops_df[stops_df['stop_name'].str.lower().str.contains('carmelram')].sort_values(by='geohashes_reached', 
                                                                                ascending=False)

In [ ]:
gh_set = []
for i in stops_df.stop_name:
    sub_df = direct_connectivity_df[direct_connectivity_df.from_stop == i].reset_index(drop=True).copy(deep=True)
    sub_gh_set = []
#     print (i)
    for j, r in sub_df.iterrows():
        sub_gh_set.extend(r['to_gh'])
#         print (sub_gh_set)
    gh_set.append(list(set(sub_gh_set)))

stops_df['direct_connect_geohashes'] = gh_set
stops_df.head(3)

In [ ]:
stops_df['geohashes_reached'] = stops_df.apply(lambda x: len(x.direct_connect_geohashes), axis=1)
stops_df.sort_values(by='geohashes_reached', ascending=False).head(5)

In [ ]:
def geohash_to_polygon(ghash):
    try:
        coords = pgh.decode_exactly(ghash)
        north = coords[0]+coords[2]
        south = coords[0]-coords[2]
        east = coords[1]+coords[3] 
        west = coords[1]-coords[3]

        return [(south, west), (north, west), 
                (north, east), (south, east), 
                (south, west)]
    except:
        print('error')
        return [(0, 0), (0, 0), 
                (0, 0), (0, 0), 
                (0, 0)]

def geohash_plot(station_name, stops_df = stops_df):
    # Create a map centered around the first geohash
    m = folium.Map(location = bengaluru_center, zoom_start = 12)
    # Plot each geohash
    for ghash in stops_df.loc[stops_df.stop_name==station_name].direct_connect_geohashes.values[0]:
        polygon = geohash_to_polygon(ghash)
        folium.Polygon(locations=polygon, color='blue', fill=True, fill_opacity=0.4, weight=0#, popup=ghash
                  ).add_to(m)
    
    folium.Marker(location=[stops_df.loc[stops_df.stop_name==station_name]['lat'].values[0], 
                            stops_df.loc[stops_df.stop_name==station_name]['lng'].values[0]], 
                  color='red', fill=True, fill_opacity=0.4, weight=0, popup=station_name
                  ).add_to(m)
    # Display the map
    return m

In [ ]:
m1 = geohash_plot('Kempegowda Bus Station')
m1

In [ ]:
m1 = geohash_plot('Malleshwara Circle')
m1

In [ ]:
m1 = geohash_plot('Banashankari Bus Station')
m1

In [ ]:
m1 = geohash_plot('Tin Factory')
m1

In [ ]:
m1 = geohash_plot('Koramangala Water Tank')
m1

In [ ]:
m1 = geohash_plot('Eco Space')
m1

In [ ]:
m1 = geohash_plot('Doddakannalli')
# Carmelram Gate
m1

In [ ]:
m1 = geohash_plot('ITPL')
m1

In [ ]:
reachable_intensity_df = stops_df[['lat', 'lng', 'geohashes_reached']][
    stops_df.geohashes_reached>0].reset_index(drop=True).copy(deep=True)
reachable_intensity_df.geohashes_reached.hist()

In [ ]:
# Plotting histogram on a log scale
import matplotlib.pyplot as plt
plt.hist(reachable_intensity_df.geohashes_reached, log=True)
plt.xlabel('Geohashes Reached')
plt.ylabel('Frequency (log scale)')
plt.title('Histogram of Geohashes Reached (log scale)')
plt.show()

In [ ]:
from folium.plugins import HeatMap

# Calculate the logarithm of the values
reachable_intensity_df['log_geohashes_reached'] = reachable_intensity_df['geohashes_reached']# np.log(reachable_intensity_df['geohashes_reached'] + 1)  # Adding 1 to avoid log(0)

# Create a map centered at the mean latitude and longitude
center_lat = reachable_intensity_df['lat'].mean()
center_lng = reachable_intensity_df['lng'].mean()
mymap = folium.Map(location=[center_lat, center_lng], zoom_start=12)

# Convert the data to a list of [lat, lng, intensity]
heat_data = [[row['lat'], row['lng'], row['log_geohashes_reached']] for index, row in reachable_intensity_df.iterrows()]

# Plot the heatmap with log scale
HeatMap(heat_data, min_opacity=0.3).add_to(mymap)

# Save the map to an HTML file
mymap.save("heatmap_log_scale.html")

# Display the map
mymap
